In [ ]:
!pip install PyPDF2
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install simplejson
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [1]:
import os
from groq import Groq

# Initialize Groq client (it will automatically read the API key from the environment variable)
client = Groq(api_key=os.environ["GROQ_API_KEY"])

it uses first matched page . If u change the threshold value to 80 You can find cola value in page 6 . It is not iterating with matched pages . limitation of liability match is found in table of contents and output is .........
limitation of liability is there in later part of document .
if iteration is done on matched pages till relevant content is found then problem is solved

In [2]:
def generate_prompt(field, page_content):
    prompts = {
        "start_date": (
            f"Extract the start date from the following text:\n\n"
            f"{page_content}\n\n"
            "Return only the date in DD-MM-YYYY format. Ignore any other dates and do not include any additional text or explanations. "
            "The start date is the date when the amendment or agreement begins. Look at the text carefully and extract only the start date of the agreement. "
            "If no start date is found, return 'NA'."
        ),
        "company_names": (
            f"Extract the company names from the following text:\n\n"
            f"{page_content}\n\n"
            "Return only the company names as a comma-separated list. No additional text or explanations. "
            "If no company names are found, return 'NA'."
        ),
        "service_provider": (
            f"Extract the name of the service provider from the following text:\n\n"
            f"{page_content}\n\n"
            "Return only the service provider name. No additional text or explanations. "
            "If no service provider name is found, return 'NA'."
        ),
        "company_address": (
            f"Extract the registered office address of the company from the following text:\n\n"
            f"{page_content}\n\n"
            "Return only the company address. The company address refers to the client's address, not the service provider's. "
            "If no company address is found, return 'NA'."
        ),
        "service_provider_address": (
            f"Extract the registered office address of the service provider from the following text:\n\n"
            f"{page_content}\n\n"
            "Return only the service provider address. No additional text or explanations. "
            "If no service provider address is found, return 'NA'."
        ),
        "cola": (
            f"Extract the Cost-of-Living Adjustment (COLA) from the following text:\n\n"
            f"{page_content}\n\n"
            "Return only the COLA amount as a percentage. No additional text or explanations. "
            "If no COLA is found, return 'NA'."
        ),
        "end_date": (
            f"Extract the end date from the following text:\n\n"
            f"{page_content}\n\n"
            "Return only the date in DD-MM-YYYY format. The end date is the date when the agreement ends.No extra explainatations only value  "
            "Do not output any other dates. If no end date is found, return 'NA'."
        ),
        "number_of_fte": (
            f"Extract the number of FTE (full-time equivalent) from the following text:\n\n"
            f"{page_content}\n\n"
            "Return only the number of FTE. No additional text or explanations. "
            "If no number of FTE is found, return 'NA'."
        ),
        "unit_rate": (
           f"Extract the unit rate from the following text:\n\n"
           f"{page_content}\n\n"
            "Return only the unit rate in the format of numbers followed by currency (e.g., '37,665 INR', '4.47 USD', '17.41 USD'). "
            "No extra text or explanations. If no valid unit rate is found, return 'NA'."
),
        "limitation_of_liability": (
            f"Extract the limitation of liability details from the following text:\n\n"
            f"{page_content}\n\n"
            "Return only the relevant limitation of liability information. No additional text or explanations."
            "check properly if u find relevant clause then only give output . dont output contenttable .... and all"
        ),

    }

    return prompts.get(field)

In [3]:
def extract_info_with_llm(field, page_content):
    prompt = generate_prompt(field, page_content)

    # Call the Groq API to get a completion
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama-3.1-8b-instant",
    )

    # Extract and return the concise response content
    response = chat_completion.choices[0].message.content
    return response.strip() if response else None


print(generate_prompt("start_date", """
    This amendment will be effective September 1, 2023, and shall remain in force for 1 year until 
    August 31, 2024. 
      The  current  rate  per  FTE  of  Rs.37,665/-  is  being  maintained  and  the  rate  effective  from 
    September 1,2023 would be as follows: 
    o  Rs.37,665/FTE/Month Inclusive of GST. 
    o  This rate is applicable to all dedicated FTE deployed on the project 
    
      Any changes to the existing headcount will be communicated in writing and timelines mutually 
    agreed. The current billable headcount deployed on the account is 5 FTE.
"""))

Extract the start date from the following text:


    This amendment will be effective September 1, 2023, and shall remain in force for 1 year until 
    August 31, 2024. 
      The  current  rate  per  FTE  of  Rs.37,665/-  is  being  maintained  and  the  rate  effective  from 
    September 1,2023 would be as follows: 
    o  Rs.37,665/FTE/Month Inclusive of GST. 
    o  This rate is applicable to all dedicated FTE deployed on the project 
    
      Any changes to the existing headcount will be communicated in writing and timelines mutually 
    agreed. The current billable headcount deployed on the account is 5 FTE.


Return only the date in DD-MM-YYYY format. Ignore any other dates and do not include any additional text or explanations. The start date is the date when the amendment or agreement begins. Look at the text carefully and extract only the start date of the agreement. If no start date is found, return 'NA'.


In [3]:
import PyPDF2
from fuzzywuzzy import fuzz



# PDF and templates setup
pdf_path = "8.pdf"
templates = {
    "start_date": [
        "amendment will be effective September 1, 2023",
        "effective from July 15, 2023",
        "rate effective from September 1, 2023",
        "agreement shall commence from",
        "starts on",
        "will be effective"
        "effective date",
        "commencement date",
        "effective from",
        "beginning on",
        "valid from",
        "entered into"
    ],
    "company_names": [
        "by and between",
        "party of the first part",
        "party of the second part",
        "entered into by",
        "entered into between",
        "this agreement is made between",
        "agreement between",
        "contract between",
        "by and among",
        "the company",
        "the corporation",
        "executed by",
        "executed on behalf of",
        "signed between"
    ],
    "service_provider": [
        "service provider",
        "the provider",
        "service agreement",
        "the provider will",
        "provider will perform"
    ],
    "company_address": [
        "registered office at",
        "principal place of business",
        "address of the company",
        "located at",
        "headquarters at",
        "corporate office at",
        "office located at",
        "having its office at"
    ],
    "service_provider_address": [
        "registered office at",
        "headquarters located at",
        "principal office at",
        "corporate office at",
        "having its registered office at "

    ],
    "cola": [
        "Cost of Living Adjustment",
        "COLA",
        "adjustment for inflation",
        "inflation adjustment",
        "cost-of-living increase"
    ],
    "end_date": [
        "end date",
        "valid date",
        "termination date",
        "expiry date",
        "contract end date",
        "agreement ends on",
        "will end on",
        "expires on",
        " 1 year until"

    ],
    "number_of_fte": [
        "deployed on account ",
        "number of FTE",
        "current headcount",
        "billable headcount",
        "FTE deployed",
        "total FTE",
        "we will deploy "
    ],
    "unit_rate" : [
      "The current rate per FTE ",
      "rate effective from",
      "/FTE/Month Inclusive of GST."
      "rate per hour",
      "rate per FTE per hr",
      "current rate per FTE",
      "unit rate",
      "Rate/hr",
      "USD per FTE",
      "USD per hr",
      "INR per FTE",
      "INR per hr",
      "existing rate valid till",
      "amended rate effective from",
],
    "limitation_of_liability" : [
      "limitation of liability",
      "aggregate liability",
      "liability shall not exceed",
      "shall not be liable beyond",
      "liability is limited to",
      "service fee paid or payable",
      "damages of any nature",
      "claims, losses, costs, damages",
      "attorney’s fees",
      "extent permitted by law",
      "liability for any and all claims",
      "liability or cause of action",
]


}

# Initialize the PDF reader and extract content from each page
with open(pdf_path, 'rb') as pdfFile:
    pdfReader = PyPDF2.PdfReader(pdfFile)
    pages = len(pdfReader.pages)

    # Store the results
    extracted_data = {"start_date": None, "company_names": None, "service_provider": None,'company_address': None, "service_provider_address": None}

    # Function to process each field and handle fallback to next page if needed
    def process_field(field):
        for page_num in range(pages):
            pageObj = pdfReader.pages[page_num]
            page_text = pageObj.extract_text()

            # Perform fuzzy matching for each template
            for template in templates[field]:
                ratio = fuzz.partial_ratio(template.lower(), page_text.lower())
                if ratio > 60:
                    print(f"Page {page_num + 1} has a match with '{template}' for {field} (Score: {ratio})")

                    # Try extracting using LLM
                    result = extract_info_with_llm(field, page_text)
                    if result:
                        print(f"Extracted {field} from Page {page_num + 1}: {result}")
                        return result  # Return the first valid result
                    else:
                        print(f"No valid {field} found on Page {page_num + 1}. Checking next page...")
                    break  # If you only need to find one match per page, break after the first match

        print(f"No valid {field} found in the document.")
        return None

    # Process each field one by one
    extracted_data["start_date"] = process_field("start_date")
    extracted_data["company_names"] = process_field("company_names")
    extracted_data["service_provider"] = process_field("service_provider")
    extracted_data["company_address"] = process_field("company_address")
    extracted_data["service_provider_address"] = process_field("service_provider_address")
    extracted_data["cola"] = process_field("cola")  # Process new field
    extracted_data["end_date"] = process_field("end_date")
    extracted_data["number_of_fte"] = process_field("number_of_fte")
    extracted_data["unit_rate"] = process_field("unit_rate")
    extracted_data["limitation_of_liability"] = process_field("limitation_of_liability")

# Final output
print("Matching and extraction completed.")
print(extracted_data)


Page 3 has a match with 'amendment will be effective September 1, 2023' for start_date (Score: 62)
Extracted start_date from Page 3: 30-09-2021
Page 1 has a match with 'by and between' for company_names (Score: 100)
Extracted company_names from Page 1: ENTRUPY INC, NEXTWEALTH ENTREPRENEURS PRIVATE LIMITED
Page 1 has a match with 'service provider' for service_provider (Score: 100)
Extracted service_provider from Page 1: NEXTWEALTH ENTREPRENEURS PRIVATE LIMITED
Page 3 has a match with 'registered office at' for company_address (Score: 100)
Extracted company_address from Page 3: The company address is:

1013 Centre Road, 
Suite 403-B, 
Wilmington, 
New Castle County, 
DE 19805, 
United States of America.
Page 3 has a match with 'registered office at' for service_provider_address (Score: 100)
Extracted service_provider_address from Page 3: GA, Alsa Glenridge,
No.32, Langford Road,
Bangalore – 560 025, India
Page 2 has a match with 'COLA' for cola (Score: 75)
Extracted cola from Page 2: NA

In [4]:
import csv
csv_file = 'extracted_fields.csv'
csv_headers = ["start_date", "company_names", "service_provider",'company_address','service_provider_address','cola',"end_date"]

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=csv_headers)
    writer.writeheader()
    writer.writerow(extracted_data)

ValueError: dict contains fields not in fieldnames: 'unit_rate', 'limitation_of_liability', 'number_of_fte'